# Center Cut Label Image

In [1]:
!pip install scikit-image

In [2]:
from itertools import groupby, chain
from PIL import Image
import matplotlib.pyplot as plt
import cv2 
import numpy as np
from skimage import io, color
import sys
from skimage import img_as_ubyte
import skimage.io as io
import os
np.set_printoptions(threshold=sys.maxsize)

### Commong Utility Functions

In [3]:
#from utilities.model_errors import ValidationError
def find_centermost_mask(input_mask, min_pixels):
    """
    Take in a binary mask and return a mask of the same shape, with only the centermost contiguous portion.
    If a mask contains no contours (objects), then just return it (it's all zeros anyway).
    """

    # Get all the bits aligned in memory
    input_mask = np.ascontiguousarray(input_mask, dtype=np.uint8)
    contours, _ = cv2.findContours(input_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

#     if not contours or len(contours) == 0:
#         output_mask = np.zeros(input_mask.shape)
#         return output_mask

    moments = [cv2.moments(contour) for contour in contours]
    
#     big_contour_area_idx = 0
#     if len(contours) > 1:
#         contour_areas = [m['m00'] for m in moments]
#         big_contour_area_idx = contour_areas.index(max(contour_areas))
#------------------------------------------------------------------    
#     (x, y) tuples of centroid locations for contours (that have area > 0)
    min_pixels = 10000
    centroids = [
        (m['m10'] / m['m00'], m['m01'] / m['m00'])
        if m['m00'] >= min_pixels else (0, 0)
        for m in moments
    ]

    if not centroids:
        raise ValidationError(f'No identified footprint contours contain more than {min_pixels} pixels.')

    # dimensions come 1-indexed, but array indicies are 0-indexed
    center = [(dim - 1) / 2 for dim in input_mask.shape]

    # Cacluate (Manhattan) distance from center of image
    distances_from_center = [abs(x - center[0]) + abs(y - center[1]) for x, y in centroids]
    centermost_contour = contours[distances_from_center.index(min(distances_from_center))]
#------------------------------------------------------------------

#     centermost_contour = contours[big_contour_area_idx]
    output_mask = np.zeros(input_mask.shape)
    cv2.fillPoly(output_mask, centermost_contour.transpose((1, 0, 2)), 1)

    return output_mask

### Center cut the Prediction Image and Center Cut the Original Image

In [4]:
label_img_path = '/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/run-1/label-626-gt'
label_cc_save_path = '/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/label-cc'

# label_img_path = '/home/ec2-user/SageMaker/data/footprint/190-test-images-label'

In [5]:
lst_label_imgs = [img for img in os.listdir(label_img_path) if os.path.isfile(os.path.join(label_img_path, img))]
lst_label_imgs[:5], len(lst_label_imgs)

(['image_386_gt.tif',
  'image_595_gt.tif',
  'image_301_gt.tif',
  'image_487_gt.tif',
  'image_87_gt.tif'],
 626)

In [ ]:
for idx, lbl_img_name in enumerate(lst_label_imgs, start=1):
    print('processing : {}, Image Name : {}'.format(idx, lbl_img_name))
    lbl_img_path_with_filename = os.path.join(label_img_path, lbl_img_name)
    lbl_gray = cv2.imread(lbl_img_path_with_filename)
    lbl_gray = cv2.cvtColor(lbl_gray, cv2.COLOR_BGR2GRAY)
    print(np.unique(lbl_gray))
    print(lbl_gray.shape)
    
#     lbl_gray[lbl_gray == 30] = 0
    unique_pixel_values = np.unique(lbl_gray)
    if(len(unique_pixel_values) > 1):  # to handle no building scenario returns one pixel value that is '0'
        _, thresh = cv2.threshold(lbl_gray, unique_pixel_values[0] + 1, 255, 0)
        print(np.unique(thresh))
        lbl_cc_img_mask=find_centermost_mask(thresh,2000)
    else:
        lbl_cc_img_mask = np.zeros(lbl_gray.shape)
    
    lbl_cc_img_mask = lbl_cc_img_mask.astype(np.byte)
    io.imsave(os.path.join(label_cc_save_path, lbl_img_name.replace('_gt', '_gt_cc')), lbl_cc_img_mask)
#     plt.imshow(lbl_cc_img_mask)
#     plt.show()

processing : 1, Image Name : image_386_gt.tif
[ 0 29]
(512, 512)
[  0 255]
processing : 2, Image Name : image_595_gt.tif
[ 0 29]
(512, 512)
[  0 255]
processing : 3, Image Name : image_301_gt.tif
[ 0 29]
(512, 512)
[  0 255]
processing : 4, Image Name : image_487_gt.tif
[ 0 29]
(512, 512)
[  0 255]
processing : 5, Image Name : image_87_gt.tif
[ 0 29]
(512, 512)
[  0 255]
processing : 6, Image Name : image_285_gt.tif
[ 0 29]
(512, 512)
[  0 255]
processing : 7, Image Name : image_354_gt.tif
[ 0 29]
(512, 512)
[  0 255]
processing : 8, Image Name : image_496_gt.tif
[ 0 29]
(512, 512)
[  0 255]
processing : 9, Image Name : image_133_gt.tif
[ 0 29]
(512, 512)
[  0 255]
processing : 10, Image Name : image_219_gt.tif


/home/ec2-user/anaconda3/envs/fastai/lib/python3.7/site-packages/ipykernel/__main__.py:19: UserWarning: /home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/label-cc/image_386_gt_cc.tif is a low contrast image
/home/ec2-user/anaconda3/envs/fastai/lib/python3.7/site-packages/ipykernel/__main__.py:19: UserWarning: /home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/label-cc/image_595_gt_cc.tif is a low contrast image
/home/ec2-user/anaconda3/envs/fastai/lib/python3.7/site-packages/ipykernel/__main__.py:19: UserWarning: /home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/label-cc/image_301_gt_cc.tif is a low contrast image
/home/ec2-user/anaconda3/envs/fastai/lib/python3.7/site-packages/ipykernel/__main__.py:19: UserWarning: /home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/label-cc/image_487_gt_cc.tif is a low contrast image
/hom

[ 0 29]
(512, 512)
[  0 255]
processing : 11, Image Name : image_449_gt.tif
[ 0 29]
(512, 512)
[  0 255]
processing : 12, Image Name : image_279_gt.tif
[ 0 29]
(512, 512)
[  0 255]
processing : 13, Image Name : image_99_gt.tif
[ 0 29]
(512, 512)
[  0 255]
processing : 14, Image Name : image_322_gt.tif
[ 0 29]
(512, 512)
[  0 255]
processing : 15, Image Name : image_143_gt.tif
[ 0 29]
(512, 512)
[  0 255]
processing : 16, Image Name : image_91_gt.tif
[ 0 29]
(512, 512)
[  0 255]
processing : 17, Image Name : image_450_gt.tif
[ 0 29]
(512, 512)
[  0 255]
processing : 18, Image Name : image_370_gt.tif
[ 0 29]
(512, 512)
[  0 255]
processing : 19, Image Name : image_492_gt.tif
[ 0 29]
(512, 512)


/home/ec2-user/anaconda3/envs/fastai/lib/python3.7/site-packages/ipykernel/__main__.py:19: UserWarning: /home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/label-cc/image_449_gt_cc.tif is a low contrast image
/home/ec2-user/anaconda3/envs/fastai/lib/python3.7/site-packages/ipykernel/__main__.py:19: UserWarning: /home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/label-cc/image_279_gt_cc.tif is a low contrast image
/home/ec2-user/anaconda3/envs/fastai/lib/python3.7/site-packages/ipykernel/__main__.py:19: UserWarning: /home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/label-cc/image_99_gt_cc.tif is a low contrast image
/home/ec2-user/anaconda3/envs/fastai/lib/python3.7/site-packages/ipykernel/__main__.py:19: UserWarning: /home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/label-cc/image_322_gt_cc.tif is a low contrast image
/home

[  0 255]
processing : 20, Image Name : image_41_gt.tif
[ 0 29]
(512, 512)
[  0 255]
processing : 21, Image Name : image_591_gt.tif
[ 0 29]
(512, 512)
[  0 255]
processing : 22, Image Name : image_406_gt.tif
[ 0 29]
(512, 512)
[  0 255]
processing : 23, Image Name : image_244_gt.tif
[ 0 29]
(512, 512)
[  0 255]
processing : 24, Image Name : image_84_gt.tif
[ 0 29]
(512, 512)
[  0 255]
processing : 25, Image Name : image_323_gt.tif
[ 0 29]
(512, 512)
[  0 255]
processing : 26, Image Name : image_134_gt.tif
[ 0 29]
(512, 512)
[  0 255]
processing : 27, Image Name : image_32_gt.tif
[ 0 29]
(512, 512)
[  0 255]
processing : 28, Image Name : image_30_gt.tif
[ 0 29]
(512, 512)
[  0 255]
processing : 29, Image Name : image_530_gt.tif


...

[ 0 29]
(512, 512)
[  0 255]
processing : 505, Image Name : image_162_gt.tif
[ 0 29]
(512, 512)
[  0 255]
processing : 506, Image Name : image_552_gt.tif
[ 0 29]
(512, 512)
[  0 255]
processing : 507, Image Name : image_356_gt.tif
[ 0 29]
(512, 512)
[  0 255]
processing : 508, Image Name : image_59_gt.tif
[ 0 29]
(512, 512)
[  0 255]
processing : 509, Image Name : image_345_gt.tif
[ 0 29]
(512, 512)
[  0 255]
processing : 510, Image Name : image_619_gt.tif
[ 0 29]
(512, 512)
[  0 255]
processing : 511, Image Name : image_533_gt.tif
[ 0 29]
(512, 512)
[  0 255]
processing : 512, Image Name : image_148_gt.tif
[ 0 29]
(512, 512)
[  0 255]
processing : 513, Image Name : image_421_gt.tif
[ 0 29]
(512, 512)
[  0 255]
processing : 514, Image Name : image_524_gt.tif


/home/ec2-user/anaconda3/envs/fastai/lib/python3.7/site-packages/ipykernel/__main__.py:19: UserWarning: /home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/label-cc/image_421_gt_cc.tif is a low contrast image
/home/ec2-user/anaconda3/envs/fastai/lib/python3.7/site-packages/ipykernel/__main__.py:19: UserWarning: /home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/label-cc/image_524_gt_cc.tif is a low contrast image
/home/ec2-user/anaconda3/envs/fastai/lib/python3.7/site-packages/ipykernel/__main__.py:19: UserWarning: /home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/label-cc/image_549_gt_cc.tif is a low contrast image
/home/ec2-user/anaconda3/envs/fastai/lib/python3.7/site-packages/ipykernel/__main__.py:19: UserWarning: /home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/label-cc/image_448_gt_cc.tif is a low contrast image
/hom

[ 0 29]
(512, 512)
[  0 255]
processing : 515, Image Name : image_549_gt.tif
[ 0 29]
(512, 512)
[  0 255]
processing : 516, Image Name : image_448_gt.tif
[ 0 29]
(512, 512)
[  0 255]
processing : 517, Image Name : image_523_gt.tif
[ 0 29]
(512, 512)
[  0 255]
processing : 518, Image Name : image_318_gt.tif
[ 0 29]
(512, 512)
[  0 255]
processing : 519, Image Name : image_383_gt.tif
[ 0 29]
(512, 512)
[  0 255]
processing : 520, Image Name : image_106_gt.tif
[ 0 29]
(512, 512)
[  0 255]
processing : 521, Image Name : image_100_gt.tif
[ 0 29]
(512, 512)
[  0 255]
processing : 522, Image Name : image_250_gt.tif
[ 0 29]
(512, 512)
[  0 255]
processing : 523, Image Name : image_494_gt.tif
[ 0 29]
(512, 512)
[  0 255]
processing : 524, Image Name : image_40_gt.tif
[ 0 29]
(512, 512)


/home/ec2-user/anaconda3/envs/fastai/lib/python3.7/site-packages/ipykernel/__main__.py:19: UserWarning: /home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/label-cc/image_494_gt_cc.tif is a low contrast image
/home/ec2-user/anaconda3/envs/fastai/lib/python3.7/site-packages/ipykernel/__main__.py:19: UserWarning: /home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/label-cc/image_40_gt_cc.tif is a low contrast image
/home/ec2-user/anaconda3/envs/fastai/lib/python3.7/site-packages/ipykernel/__main__.py:19: UserWarning: /home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/label-cc/image_25_gt_cc.tif is a low contrast image
/home/ec2-user/anaconda3/envs/fastai/lib/python3.7/site-packages/ipykernel/__main__.py:19: UserWarning: /home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/label-cc/image_124_gt_cc.tif is a low contrast image
/home/

[  0 255]
processing : 525, Image Name : image_25_gt.tif
[ 0 29]
(512, 512)
[  0 255]
processing : 526, Image Name : image_124_gt.tif
[ 0 29]
(512, 512)
[  0 255]
processing : 527, Image Name : image_574_gt.tif
[ 0 29]
(512, 512)
[  0 255]
processing : 528, Image Name : image_211_gt.tif
[ 0 29]
(512, 512)
[  0 255]
processing : 529, Image Name : image_557_gt.tif
[ 0 29]
(512, 512)
[  0 255]
processing : 530, Image Name : image_65_gt.tif
[ 0 29]
(512, 512)
[  0 255]
processing : 531, Image Name : image_45_gt.tif
[ 0 29]
(512, 512)
[  0 255]
processing : 532, Image Name : image_511_gt.tif
[ 0 29]
(512, 512)
[  0 255]
processing : 533, Image Name : image_361_gt.tif
[ 0 29]
(512, 512)


### Backups code  - no need to run 

In [ ]:
#from utilities.model_errors import ValidationError
def find_centermost_mask_bkup(input_mask, min_pixels):
    """
    Take in a binary mask and return a mask of the same shape, with only the centermost contiguous portion.
    If a mask contains no contours (objects), then just return it (it's all zeros anyway).
    """

    # Get all the bits aligned in memory
    input_mask = np.ascontiguousarray(input_mask, dtype=np.uint8)
    contours, _ = cv2.findContours(input_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not contours or len(contours) == 1:
        return input_mask

    moments = [cv2.moments(contour) for contour in contours]
    # (x, y) tuples of centroid locations for contours (that have area > 0)
    centroids = [
        (m['m10'] / m['m00'], m['m01'] / m['m00'])
        if m['m00'] and m['m00'] >= min_pixels else (0, 0)
        for m in moments
    ]

    if not centroids:
        raise ValidationError(f'No identified footprint contours contain more than {min_pixels} pixels.')

    # dimensions come 1-indexed, but array indicies are 0-indexed
    center = [(dim - 1) / 2 for dim in input_mask.shape]

    # Cacluate (Manhattan) distance from center of image
    distances_from_center = [abs(x - center[0]) + abs(y - center[1]) for x, y in centroids]
    centermost_contour = contours[distances_from_center.index(min(distances_from_center))]

    output_mask = np.zeros(input_mask.shape)
    cv2.fillPoly(output_mask, centermost_contour.transpose((1, 0, 2)), 1)

    return output_mask